## T. Martz-Oberlander, 2015-11-12, CO2 and Speed of Sound
### Formatting ENVIRONMENTAL CONDITIONS pipe organ data for Python operations



The entire script looks for mathematical relationships between CO2 concentration changes and pitch changes from a pipe organ. This script uploads, cleans data and organizes new dataframes, creates figures, and performs statistical tests on the relationships between variable CO2 and frequency of sound from a note played on a pipe organ.

This uploader script:

1) Uploads CO2, temp, and RH data files

2) Munges it (creates a Date Time column for the time stamps), establishes column contents as floats

[  Here I pursue data analysis route 1 (as mentionted in my notebook.md file), which involves comparing one pitch dataframe with one dataframe of environmental characteristics taken at one sensor location. Both dataframes are compared by the time of data recorded. ]





In [1]:
# I import useful libraries (with functions) so I can visualize my data
# I use Pandas because this dataset has word/string column titles and I like the readability features of commands and finish visual products that Pandas offers

import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

%matplotlib inline

#I want to be able to easily scroll through this notebook so I limit the length of the appearance of my dataframes 
from pandas import set_option
set_option('display.max_rows', 10)


##Uploaded data into Python¶


First I upload my data set(s). I am working with environmental data from different locations in the church at differnet dates. Files include: environmental characteristics (CO2, temperature (deg C), and relative humidity (RH) (%) measurements). 

In [9]:
#I import a pitch data file

#comment by nick changed the path you upload that data from making in compatible with clone copies of your project
env_choir_div=pd.read_table('../Data/Choir_Division_May.csv', sep=',')

#assigning columns names
env_choir_div.columns=[['test', 'date_time','temp C', 'RH %']]

#I display my dataframe
env_choir_div

,test,date_time,temp C,RH %
0,1,04/13/10 12:00:00 PM,22.142,30.78
1,2,04/13/10 12:01:00 PM,22.046,30.88
2,3,04/13/10 12:02:00 PM,21.951,30.97
3,4,04/13/10 12:03:00 PM,21.951,31.10
4,5,04/13/10 12:04:00 PM,21.855,31.19
...,...,...,...,...
26066,26067,2005-01-10 14:26,20.805,38.65
26067,26068,2005-01-10 14:27,20.805,38.61
26068,26069,2005-01-10 14:28,20.805,38.61
26069,26070,2005-01-10 14:29,20.805,38.61


In [10]:
#comment by nick changing your data time variable to actual values of time. 
env_choir_div['date_time']= pd.to_datetime(env_choir_div['date_time'])

#print the new table and the type of data. 
print(env_choir_div)

env_choir_div.dtypes

        test           date_time  temp C   RH %
0          1 2010-04-13 12:00:00  22.142  30.78
1          2 2010-04-13 12:01:00  22.046  30.88
2          3 2010-04-13 12:02:00  21.951  30.97
3          4 2010-04-13 12:03:00  21.951  31.10
4          5 2010-04-13 12:04:00  21.855  31.19
...      ...                 ...     ...    ...
26066  26067 2005-01-10 14:26:00  20.805  38.65
26067  26068 2005-01-10 14:27:00  20.805  38.61
26068  26069 2005-01-10 14:28:00  20.805  38.61
26069  26070 2005-01-10 14:29:00  20.805  38.61
26070  26071 2005-01-10 14:30:00  20.805  38.61

[26071 rows x 4 columns]


test                  int64
date_time    datetime64[ns]
temp C              float64
RH %                float64
dtype: object

##Next
###1. Find the average pitch value for each date_time
###2. Select out the pitch data for one division at a time. Make an argument
###3. Append other pitch files¶


In [ ]:
#make a loop 

for line in 